In [1]:
import warnings
import math

import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    model_ready_data_paths,
    get_derived_csv_path,
    get_derived_lpa_path,
    get_derived_plps_path,
)

from jre_utils.process import get_most_active_municipalities

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [3]:
asset_type = "building"
years_ahead = 2

metrics = {
    "weighted_mean": "unit_price_wmean",
    "weighted_median": "unit_price_wmedian",
    "mean": "unit_price_mean",
    "median": "unit_price_median",
    "weighted_mean_smoothed": "unit_price_wmean_smoothed",
    "weighted_median_smoothed": "unit_price_wmedian_smoothed",
    "mean_smoothed": "unit_price_mean_smoothed",
    "median_smoothed": "unit_price_median_smoothed",
}

dataset_paths = {
    "transactions": get_derived_csv_path(asset_type),
    "lpa": get_derived_lpa_path(),
    "plps": get_derived_plps_path()
}

granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

metric_key = "weighted_median_smoothed"
metric = metrics[metric_key]
metric_pct_chg = metric + "_pct_chg"
upcoming_metric = "upcoming_" + metric

In [4]:
dataset_key = "transactions"
core_path = dataset_paths[dataset_key]
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"]["municipality"]


In [5]:
def years_since_crisis(year):
    year_ranges = {
        (1960, 1973): 1960,
        (1973, 1990): 1973,
        (1990, 1997): 1990,
        (1997, 2008): 1997,
        (2008, 2019): 2008,
        (2019, math.inf): 2019,
    }

    for range_start, range_end in year_ranges:
        if range_start <= year < range_end:
            return year - year_ranges[(range_start, range_end)]
        

In [6]:
df = pd.read_csv(core_path)
df = get_most_active_municipalities(df, 1500)

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

In [7]:
df["migrations_is_available"] = df["net_migration_ratio"].notnull().astype(int)
df["taxable_income_is_available"] = df["taxable_income"].notnull().astype(int)
df["total_tax_is_available"] = df["total_tax"].notnull().astype(int)

# Might go back and undo the new dwellings filling for unknown municipalities.
df["dwellings_is_available"] = df["new_dwellings"].notnull().astype(int)

# Years since crisis (Yn - 2008) or (Yn - 2020)
df["years_since_crisis"] = df["year"].apply(years_since_crisis)

df = df.fillna(0)
# df

In [8]:
# prepare metrics
df = df.sort_values(by=group_by_columns, ascending=True)
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(periods=years_ahead)

# time box
# start_year = 2005
# end_year = 2023

# df = df[(df["year"] >= start_year) & (df["year"] <= end_year)]

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25315 entries, 16845 to 14370
Data columns (total 31 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   year                                 25315 non-null  int64  
 1   area_code                            25315 non-null  int64  
 2   area                                 25315 non-null  object 
 3   unit_price_wmean                     25315 non-null  float64
 4   unit_price_wmedian                   25315 non-null  float64
 5   unit_price_mean                      25315 non-null  float64
 6   unit_price_median                    25315 non-null  float64
 7   total_traded_area                    25315 non-null  float64
 8   count                                25315 non-null  float64
 9   unit_price_wmean_smoothed            25315 non-null  float64
 10  unit_price_wmedian_smoothed          25315 non-null  float64
 11  unit_price_mean_smoothed     

In [10]:
df.columns

Index(['year', 'area_code', 'area', 'unit_price_wmean', 'unit_price_wmedian',
       'unit_price_mean', 'unit_price_median', 'total_traded_area', 'count',
       'unit_price_wmean_smoothed', 'unit_price_wmedian_smoothed',
       'unit_price_mean_smoothed', 'unit_price_median_smoothed', 'population',
       'net_migration_ratio', 'taxable_income', 'taxpayer_count',
       'taxable_income_per_taxpayer', 'taxable_income_growth',
       'taxable_income_per_taxpayer_growth', 'new_dwellings',
       'existing_dwellings', 'new_dwellings_ratio', 'total_tax',
       'total_tax_growth', 'migrations_is_available',
       'taxable_income_is_available', 'total_tax_is_available',
       'dwellings_is_available', 'years_since_crisis',
       'unit_price_wmedian_smoothed_pct_chg'],
      dtype='object')

In [12]:
columns = [
    metric_pct_chg,
    metric,
    "year",
    "years_since_crisis",
    "count",
    "total_traded_area",
    "population",
    "taxpayer_count",
    "taxable_income",
    "taxable_income_per_taxpayer",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "total_tax",
    "total_tax_growth",
    "new_dwellings",
    "existing_dwellings",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "migrations_is_available",
    "taxable_income_is_available",
    "dwellings_is_available",
    "total_tax_is_available",
    "area_code",  # id
]

# add ratios and growths if necessary

df = df[columns]
df = df.dropna()

In [13]:
print(f"Initial Size: ", df.shape[0])
q = 0.01
filtered_df = df.copy()
filter_col = metric_pct_chg
filtered_df = filtered_df[
    (filtered_df[filter_col] >= filtered_df[filter_col].quantile(q))
    & (filtered_df[filter_col] <= filtered_df[filter_col].quantile(1 - q))
]
print(f"Filtered Size: ", filtered_df.shape[0])
filtered_df.describe()

Initial Size:  22315
Filtered Size:  21867


,unit_price_wmedian_smoothed_pct_chg,unit_price_wmedian_smoothed,year,years_since_crisis,count,total_traded_area,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,total_tax,total_tax_growth,new_dwellings,existing_dwellings,net_migration_ratio,new_dwellings_ratio,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,area_code
count,21867.000000,2.186700e+04,21867.000000,21867.000000,21867.000000,21867.000000,2.186700e+04,2.186700e+04,2.186700e+04,21867.000000,21867.000000,21867.000000,2.186700e+04,21867.000000,21867.000000,2.186700e+04,21867.000000,21867.000000,21867.000000,21867.000000,21867.000000,21867.000000,21867.000000
mean,-0.000593,7.208540e+04,2015.890794,4.354415,78.132803,20714.112818,8.304147e+04,3.521177e+04,1.181324e+08,2522.167423,0.002475,0.002178,1.289725e+07,0.003247,531.916770,3.423459e+04,-0.003072,0.006383,0.937852,0.873417,0.488636,0.940047,21467.543559
std,0.273908,1.962551e+05,4.321446,3.028851,198.863859,39885.026520,2.055271e+05,9.243731e+04,3.471286e+08,1095.047512,0.041031,0.031123,4.052008e+07,0.061631,1794.725477,1.016387e+05,0.006989,0.007971,0.241430,0.332513,0.499882,0.237406,13798.652884
min,-0.621123,1.025209e+03,2007.000000,0.000000,1.000000,65.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.537047,-0.480760,0.000000e+00,-0.677251,0.000000,0.000000e+00,-0.313065,0.000000,0.000000,0.000000,0.000000,0.000000,1100.000000
25%,-0.167355,1.365116e+04,2012.000000,2.000000,9.000000,3907.500000,1.112650e+04,3.557500e+03,9.233596e+06,2457.305510,-0.008292,-0.006307,1.372152e+06,-0.014501,0.000000,0.000000e+00,-0.006570,0.000000,1.000000,1.000000,0.000000,1.000000,10344.000000
50%,-0.027596,2.676741e+04,2016.000000,4.000000,24.000000,9375.000000,2.992700e+04,1.130500e+04,3.040271e+07,2718.857673,0.001458,0.000204,3.749085e+06,0.000000,0.000000,0.000000e+00,-0.002818,0.000000,1.000000,1.000000,0.000000,1.000000,20521.000000
75%,0.119323,6.694880e+04,2020.000000,7.000000,70.000000,21392.500000,7.357050e+04,2.991050e+04,8.984681e+07,3028.263334,0.018424,0.011466,1.038226e+07,0.017280,384.000000,2.970550e+04,0.000000,0.012158,1.000000,1.000000,1.000000,1.000000,32449.000000
max,1.272794,6.241887e+06,2023.000000,10.000000,4358.000000,666150.000000,3.794024e+06,1.906224e+06,7.965148e+09,12667.020000,1.066040,1.026488,8.672765e+08,4.759148,39143.000000,1.916062e+06,0.099253,0.120270,1.000000,1.000000,1.000000,1.000000,47362.000000


In [14]:
filtered_df.to_csv(model_ready_data_paths[f"sequence_{dataset_key}_{metric_key}_{years_ahead}"], index=False)

In [15]:
filtered_df[filtered_df["area_code"] == 13101][["year", metric, metric_pct_chg]]

,year,unit_price_wmedian_smoothed,unit_price_wmedian_smoothed_pct_chg
2012,2007,3.190579e+06,0.121657
2013,2008,3.144827e+06,0.055103
2014,2009,2.910509e+06,-0.087780
2015,2010,2.697234e+06,-0.142327
2016,2011,2.354779e+06,-0.190939
2017,2012,2.224987e+06,-0.175086
2018,2013,2.791684e+06,0.185540
2019,2014,3.167834e+06,0.423754
2020,2015,3.297759e+06,0.181280
2021,2016,3.560900e+06,0.124081
